# Meshing

## Inviscid Wedge
![Inviscid Wedge](../assets/inviscid_wedge.png)

https://su2code.github.io/tutorials/Inviscid_Wedge/

# Method 1: Manually define lines and points

In [3]:
from typing import List
from ezmesh import CurveLoop, PlaneSurface, TransfiniteSurfaceField, Geometry, Point, Line, TransfiniteCurveField, visualize_mesh
from ezmesh.geometry import SegmentType
import gmsh

with Geometry() as geo:
    mesh_size = 0.05

    points = [
        Point([0, 1], mesh_size),
        Point([1.5, 1], mesh_size),
        Point([1.5, 0.2], mesh_size),
        Point([0.5, 0], mesh_size),
        Point([0, 0], mesh_size),
    ]

    segments: List[SegmentType] = [
        Line(start=points[0], end=points[1], label="upper"),
        Line(start=points[1], end=points[2], label="outlet"),
        Line(start=points[2], end=points[3], label="lower/1"),
        Line(start=points[3], end=points[4], label="lower/2"),
        Line(start=points[4], end=points[0], label="inlet"),
    ]

    transfinite_field = TransfiniteCurveField(node_counts=[150,200,100,50,200])

    wedge_curve_loop = CurveLoop(segments, fields=[transfinite_field])
    surface = PlaneSurface(
        outlines=[wedge_curve_loop],
        is_quad_mesh=True,
        fields=[
            TransfiniteSurfaceField([wedge_curve_loop.points[i] for i in [0, 1, 2, 4]])
        ]
    )
    gmsh.model.geo.add_point(2,2,2,0.1)

    mesh = geo.generate(surface)
    visualize_mesh(mesh)
    geo.write("mesh_wedge_inv.su2")

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 20%] Meshing curve 2 (Line)
Info    : [ 40%] Meshing curve 3 (Line)
Info    : [ 60%] Meshing curve 4 (Line)
Info    : [ 80%] Meshing curve 5 (Line)
Info    : Done meshing 1D (Wall 0.000687963s, CPU 0.001159s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Transfinite)
Info    : Done meshing 2D (Wall 0.00405129s, CPU 0.004092s)
Info    : Meshing 3D...
Info    : Done meshing 3D (Wall 1.1631e-05s, CPU 1.1e-05s)
Info    : 30352 nodes 30706 elements


HTML(value='Coords: ()')

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, far=100000.0, near=0.001, position=(0.0, 0.0, 1.0…

,upper
,inlet
,lower
,outlet
,Zone 0


## Method 2: Automatically generate points and lines

In [4]:
from ezmesh import CurveLoop, PlaneSurface, TransfiniteSurfaceField, TransfiniteCurveField, Geometry, visualize_mesh
import numpy as np

with Geometry() as geo:
    wedge_coords = np.array([[0, 1], [1.5, 1], [1.5, 0.2], [0.5, 0], [0, 0]])
    
    wedge_curve_loop = CurveLoop.from_coords(
        wedge_coords, 
        mesh_size = 0.05,
        curve_labels=["upper", "outlet", "lower/1", "lower/2", "inlet"],
        fields=[
           TransfiniteCurveField(node_counts=[150,200,100,50,200])
        ]
    )
    surface = PlaneSurface(
        outlines=[wedge_curve_loop],
        is_quad_mesh=True,
        fields=[
            TransfiniteSurfaceField(corners=[*wedge_curve_loop.get_points("upper"), *wedge_curve_loop.get_points("lower")])

        ]
    )
    mesh = geo.generate(surface)
    visualize_mesh(mesh)
    geo.write("mesh_wedge_inv.su2")

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 20%] Meshing curve 2 (Line)
Info    : [ 40%] Meshing curve 3 (Line)
Info    : [ 60%] Meshing curve 4 (Line)
Info    : [ 80%] Meshing curve 5 (Line)
Info    : Done meshing 1D (Wall 0.000637603s, CPU 0.001187s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Transfinite)
Info    : Done meshing 2D (Wall 0.00869738s, CPU 0.007926s)
Info    : Meshing 3D...
Info    : Done meshing 3D (Wall 1.5447e-05s, CPU 1.4e-05s)
Info    : 30351 nodes 30705 elements


HTML(value='Coords: ()')

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, far=100000.0, near=0.001, position=(0.0, 0.0, 1.0…

,upper
,inlet
,lower
,outlet
,Zone 0
